# Efficient hyperparameter search with GridSearchCV and optuna

Authors:
- [Alexandre Gramfort](https://alexandre.gramfort.net/)
- [Thomas Moreau](https://tommoral.github.io/about.html)
- [Pedro L. C. Rodrigues](https://plcrodrigues.github.io/)

adapted from the work of Olivier Grisel and Andreas Mueller.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

Most models have hyperparameters that influence the complexity of the functions that they can learn. Think the depth of a decision tree.

<img src="figures/overfitting_underfitting_cartoon.svg" width="60%">

## Hyperparameters, Over-fitting, and Under-fitting

Unfortunately, there is no general rule on how to find the sweet spot, and so machine learning practitioners have to find the best trade-off of model-complexity and generalization by trying several hyperparameter settings. Hyperparameters are the internal knobs or tuning parameters of a machine learning algorithm (in contrast to model parameters that the algorithm learns from the training data -- for example, the weight coefficients of a linear regression model); the depth of a decision tree or the number of trees in a gradient boosting are such hyperparameters.

Most commonly this "hyperparameter tuning" is done using a brute force search, for example over multiple values of ``max_depth``:

In [ ]:
import pandas as pd

from sklearn.model_selection import cross_val_score, KFold
from sklearn.tree import DecisionTreeRegressor

print('Loading data...')
# load or create your dataset
df_train = pd.read_csv('datasets/regression.train', header=None, sep='\t')
df_test = pd.read_csv('datasets/regression.test', header=None, sep='\t')

df = pd.concat([df_train, df_test], axis=0)
y = df[0].values
X = df.drop(0, axis=1).values

cv = KFold(shuffle=True, n_splits=5, random_state=42)

reg = DecisionTreeRegressor()

# for each parameter setting do cross-validation:
for max_depth in [1, 3, 5, 10, 20]:
    reg.set_params(max_depth=max_depth)
    scores = cross_val_score(reg, X, y, cv=cv, scoring="r2")
    print(f"max_depth: {max_depth}, average score: {np.mean(scores)}")

There is a function in scikit-learn, called ``validation_plot`` to reproduce the cartoon figure above. It plots one parameter, such as the number of neighbors, against training and validation error (using cross-validation):

In [ ]:
from sklearn.model_selection import validation_curve

max_depth = np.arange(1, 20)
    
train_scores, test_scores = validation_curve(
    reg, X, y, param_name="max_depth",
    param_range=max_depth, cv=cv,
    scoring="r2"
)
plt.plot(max_depth, train_scores.mean(axis=1), label="train R2")
plt.plot(max_depth, test_scores.mean(axis=1), label="test R2")
plt.ylabel('R2')
plt.xlabel('Tree depth')
plt.xlim([1, max(max_depth)])
plt.legend(loc="best");

One way to automatize hyperparameter search is to use a built-in class in scikit-learn: ``GridSearchCV``, which takes a dictionary describing the parameters to sweep and which model to train. This grid of parameters is defined as a dictionary, where the keys are the parameters and the values are the settings to be tested.

To inspect the training score on different folds, we can set parameter ``return_train_score`` to ``True``.

In [ ]:
from sklearn.model_selection import GridSearchCV
max_depth = [1, 2, 3, 4, 5, 6, 7]
param_grid = {'max_depth': max_depth}

grid = GridSearchCV(DecisionTreeRegressor(), param_grid=param_grid,
                    cv=cv, verbose=3,
                    return_train_score=True, n_jobs=-1)

One of the great things about `GridSearchCV` is that it is a **meta-estimator**. It takes an estimator like `DecisionTreeRegressor` above, and creates a new estimator that behaves exactly the same - in this case, like a regressor.
So we can call ``fit`` on it, to train it:

In [ ]:
grid.fit(X, y)

What ``fit`` does is a bit more involved then what we did above. First, it runs the same loop with cross-validation, to find the best parameter combination.
Once it has the best combination, it runs fit again on all data passed to fit (without cross-validation), to build a single new model using the best parameter setting.

> [PLCR] In other words, is does a sub-cross validation using smaller splits for X

Then, as with all models, we can use ``predict`` or ``score``:


In [ ]:
grid.predict(X)

You can inspect the best parameters found by ``GridSearchCV`` in the ``best_params_`` attribute, and the best score in the ``best_score_`` attribute:

In [ ]:
print(grid.best_score_)

In [ ]:
print(grid.best_params_)

We can investigate the performance and much more for each set of parameter values by accessing the `cv_results_` attributes, which is a dictionary where each key is a string and each value is an array. It can therefore be used to make a pandas DataFrame.

In [ ]:
type(grid.cv_results_)

In [ ]:
print(grid.cv_results_.keys())

In [ ]:
import pandas as pd

cv_results = pd.DataFrame(grid.cv_results_)
cv_results.head()

In [ ]:
cv_results_tiny = cv_results[['param_max_depth', 'mean_test_score']]
cv_results_tiny.sort_values(by='mean_test_score', ascending=False).head()

There is a problem with using this score for evaluation, however. You might be making what is called a **multiple hypothesis testing error**: if you try several parameter settings, some of them will work better just by chance, and the score that you have obtained might not reflect how your model would perform on new unseen data.
Therefore, it is good to split off a separate test-set before performing grid-search. This pattern can be seen as a training-validation-test split, and is common in machine learning:

<img src="figures/grid_search_cross_validation.svg" width="70%">

We can do this very easily by splitting of some test data using ``train_test_split``, training ``GridSearchCV`` on the training set, and applying the ``score`` method to the test set:

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

param_grid = {'max_depth': max_depth}
cv = KFold(n_splits=10, shuffle=True)

grid = GridSearchCV(DecisionTreeRegressor(), param_grid=param_grid, cv=cv)

grid.fit(X_train, y_train)
grid.score(X_test, y_test)

We can also look at the parameters that were selected:

In [ ]:
grid.best_params_

Some practitioners go for an easier scheme, splitting the data simply into three parts, training, validation and testing. This is a possible alternative if your training set is very large, or it is infeasible to train many models using cross-validation because training a model takes very long.
You can do this with scikit-learn for example by splitting of a test-set and then applying GridSearchCV with ShuffleSplit cross-validation with a single iteration:

<img src="figures/train_validation_test2.svg" width="60%">

In [ ]:
from sklearn.model_selection import train_test_split, ShuffleSplit

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

param_grid = {'max_depth': max_depth}
single_split_cv = ShuffleSplit(n_splits=1, test_size=0.2)

grid = GridSearchCV(DecisionTreeRegressor(), param_grid=param_grid, cv=single_split_cv, verbose=3)

grid.fit(X_train, y_train)
grid.score(X_test, y_test)

This is much faster, but it might result in worse hyperparameters and therefore worse results as the error estimate on left-out data will have much more variance. In other words you're more likely to be unlucky!

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
      <li>
      Apply grid-search to find the best learning_rate and number of trees in a HistGradientBoostingRegressor.
      </li>
    </ul>
</div>

Solution is in: `solutions/03-gbdt_grid_search_cv.py`

Suppose now that we wanted to tune the hyperparameters of a regressor coming out from `skrub` such as in:

In [ ]:
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
est = make_pipeline(StandardScaler(), HistGradientBoostingRegressor())
est

To access the `learning_rate` from the regressor in this pipeline, we need help from `get_params` attribute

In [ ]:
est.get_params()

In [ ]:
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler
import pandas as pd

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

param_grid = {'histgradientboostingregressor__learning_rate': [0.1, 0.5, 1.0]}
single_split_cv = ShuffleSplit(n_splits=1, test_size=0.2)

grid = GridSearchCV(est, param_grid=param_grid, cv=single_split_cv, verbose=3)

grid.fit(X_train, y_train)
grid.score(X_test, y_test)

**What did Grid Search actually do?**

At this point, it is useful to pause and reflect:

- How many models were trained in total?
- How many of them performed clearly worse than the best one?
- Did the grid resolution match the sensitivity of the model?

Grid search does not *learn* from previous evaluations.
Each configuration is tested independently, even if previous results already suggest it is unlikely to perform well.

This observation motivates a more adaptive strategy.

## Guided hyper-optimization

So far we did hyper-optimization by giving a list of values to be tried. 

We were able to automatically generated these values (randomly or not) by using `RandomSearchCV` or `GridSearchCV`.

We can expect to do better by trying certain parameters which "are more likely" to optimize our problem given previous parameters.

We will use `optuna` to do so.


URL : https://pypi.org/project/optuna/

See video : https://www.youtube.com/watch?v=J_aymk4YXhg

In [ ]:
!pip install optuna

In [ ]:
import optuna

def objective(trial):
    x = trial.suggest_float('x', -10, 10)
    return (x-2)**2

study = optuna.create_study()
study.optimize(objective, n_trials=100)

print(study.best_params)

In [ ]:
import optuna
from optuna import samplers
from sklearn.svm import SVR

def objective(trial):

    df_train = pd.read_csv('datasets/regression.train', header=None, sep='\t')
    df_test = pd.read_csv('datasets/regression.test', header=None, sep='\t')
    df = pd.concat([df_train, df_test], axis=0)
    y = df[0].values
    X = df.drop(0, axis=1).values
    
    cat = trial.suggest_categorical("est", ["boost", "svr"])
    
    if cat == 'boost':
    
        max_depth = trial.suggest_int('max_depth', 2, 32)
        learning_rate = trial.suggest_float('learning_rate', 10**-5, 10**0, log=True)
        l2_regularization = trial.suggest_float('l2_regularization', 10**-5, 10**0, log=True)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 100)

        reg = HistGradientBoostingRegressor(
            max_depth=max_depth,
            learning_rate=learning_rate,
            l2_regularization=l2_regularization, 
            min_samples_leaf=min_samples_leaf,
            random_state=42,
        )

    elif cat == 'svr':

        C = trial.suggest_float('C', 1e-3, 1e1, log=True)
        reg = SVR(C=C)
        
    return np.mean(cross_val_score(reg, X, y, cv=5, n_jobs=-1, scoring="r2"))

sampler = samplers.TPESampler(seed=10)
study = optuna.create_study(sampler=sampler, direction='maximize')
optuna.logging.disable_default_handler()  # limit verbosity
study.optimize(objective, n_trials=10)

# Show best result
print(study.best_trial.params)
print(study.best_trial.value)

In [ ]:
values = [t.value for t in study.trials]
plt.plot(values)

In [ ]:
values = [t.value for t in study.trials]
values = [np.max(values[:k]) for k in range(1, len(values))]
plt.plot(values)
plt.xlabel('Trials')
plt.ylabel('R2')

In [ ]:
reg = HistGradientBoostingRegressor(random_state=42)
params = study.best_trial.params
params.pop('est')
reg.set_params(**params)
reg.fit(X_train, y_train)
reg.score(X_test, y_test)

To learn more please have a look at the doc of optuna. In particular:

https://optuna.readthedocs.io/en/stable/reference/generated/optuna.study.Study.html#optuna.study.Study.optimize